#Домашняя работа № 3. Гибридные рекомендательные системы

##Набор данных MovieLens объемом 100 тыс.

Посмотрим на оригинальный датасет

In [20]:
import pandas as pd
import numpy as np

In [17]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

--2025-02-01 11:45:18--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  8.96MB/s    in 0.5s    

2025-02-01 11:45:19 (8.96 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [22]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [ ]:
# прочитаем описание
"""
ПОДРОБНЫЕ ОПИСАНИЯ ФАЙЛОВ ДАННЫХ
==============================================

Вот краткое описание данных.

ml-data.tar.gz -- Сжатый файл tar.  Чтобы восстановить файлы данных u, выполните следующее:
                gunzip ml-data.tar.gz
                tar xvf ml-data.tar
                mku.sh

u.data - Полный набор u-данных, 100 000 оценок 943 пользователями по 1682 фильмам.
         Каждый пользователь поставил оценку как минимум 20 фильмам. Пользователи и материалы
         нумеруются последовательно, начиная с 1.  Данные получены случайным образом.
         Это разделенный табуляцией список идентификаторов
         пользователей | элементов | оценок | временных меток.
         Временные метки указаны в секундах с 1.01.1970 UTC в системе unix

u.info - Количество пользователей, элементов и оценок в наборе данных u.

u.item - Информация о фильмах, которые будут показаны в этом разделе;
         список фильмов, которые будут показаны в виде вкладок:
         идентификатор фильма | название фильма | дата выхода | дата выхода видео |
         URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
         Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
         Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
         Триллер | Военный | Вестерн |
         Последние 19 полей обозначают жанры, 1 указывает на фильм относится к этому жанру,
         0 означает, что это не так; фильмы могут быть в сразу в нескольких жанрах.
         Идентификаторы фильмов используются в наборе данных u.data.

u.genre - список жанров.

u.user - Демографическая информация о пользователях; это вкладка разделенный список
         идентификаторов пользователей
         | возраст | пол | род занятий | почтовый индекс.
         Идентификаторы пользователей используются в наборе данных u.data.

u.occupation - Список профессий.

u1.base - Наборы данных u1.base и u1.test через u5.base и u5.test
u1.test представляют собой 80% /20% разбиений данных u на обучающие и тестовые данные.
u2.base Каждый из наборов тестов u1, ..., u5 имеет непересекающиеся наборы тестов;
        это если для
u2.test 5-кратной перекрестной проверки (где вы повторяете свой эксперимент
        u3.base для каждого обучающего и тестового набора и усредняете результаты).
u3.test Эти наборы данных могут быть сгенерированы из u.data с помощью mku.sh.
u4.base
u4.test
u5.base
u5.test

ua.base - Наборы данных ua.base, ua.test, ub.base и ub.test
ua.test разделяют данные u на обучающий набор и тестовый набор с
ub.base ровно по 10 оценок для каждого пользователя в тестовом наборе.  Наборы
ub.test ua.test и ub.test не пересекаются.  Эти наборы данных могут
быть сгенерированы из u.data с помощью mku.sh.

allbut.pl -- Скрипта, который генерирует обучающие и тестовые наборы, где
             все оценки пользователей, кроме n, содержатся в обучающих данных.

mku.sh -- Командный скрипт для генерации всех u наборов данных из u.data.

"""

In [25]:
# прочитаем
u_data = pd.read_csv('/content/unpacked_files/ml-100k/u.data')
u_info = pd.read_csv('/content/unpacked_files/ml-100k/u.info')
u_item = pd.read_csv('/content/unpacked_files/ml-100k/u.item')
u_genre = pd.read_csv('/content/unpacked_files/ml-100k/u.genre')
u_user = pd.read_csv('/content/unpacked_files/ml-100k/u.user')
u_occupation = pd.read_csv('/content/unpacked_files/ml-100k/u.occupation')

In [27]:
# пользователm | элемент | оценка | временная метка
u_data.head()

,196\t242\t3\t881250949
0,186\t302\t3\t891717742
1,22\t377\t1\t878887116
2,244\t51\t2\t880606923
3,166\t346\t1\t886397596
4,298\t474\t4\t884182806


In [ ]:
u_info.head()

In [ ]:
# идентификатор фильма | название фильма | дата выхода | дата выхода видео |
# URL на IMDb | неизвестен | Боевик | Приключения | Анимация |
# Детский | Комедийный | Криминальный | Документальный | Драма | фэнтези |
# Фильм-Нуар | Ужасы | Мюзикл | Мистика | Мелодрама | Научная фантастика |
# Триллер | Военный | Вестерн |
u_item.head()

In [ ]:
u_genre.head()

In [ ]:
# возраст | пол | род занятий | почтовый индекс
u_user.head()

In [ ]:
u_occupation.head()

In [4]:
import numpy as np

In [7]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831162 sha256=e2560cb51de6b1644deccc284901cb5cb24b22fecf0d83b8f92d61b69b605efe
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [8]:
from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [9]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [10]:
train = movielens['train']
test = movielens['test']

In [16]:
train.toarray()

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM()
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.43, test 0.13.


In [ ]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.61, test 0.23.


In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()
                          [user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

In [ ]:
sample_recommendation(model, movielens, [10, 25, 451])

User 10
     Known positives:
        Babe (1995)
        Dead Man Walking (1995)
        Seven (Se7en) (1995)
     Recommended:
        Raiders of the Lost Ark (1981)
        Star Wars (1977)
        Return of the Jedi (1983)
User 25
     Known positives:
        Toy Story (1995)
        Twelve Monkeys (1995)
        Dead Man Walking (1995)
     Recommended:
        Scream (1996)
        Contact (1997)
        Fargo (1996)
User 451
     Known positives:
        Twelve Monkeys (1995)
        Babe (1995)
        Postino, Il (1994)
     Recommended:
        Raiders of the Lost Ark (1981)
        Amadeus (1984)
        Star Wars (1977)
